In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
##Reading the features and responder csv files to understand the dataset
# Load the dataset
features_data = pd.read_csv('/kaggle/input/jane-street-real-time-market-data-forecasting/features.csv')
responders_data =  pd.read_csv('/kaggle/input/jane-street-real-time-market-data-forecasting/responders.csv')
# Display the first few rows of the dataset


In [ ]:
#Get the length of the features_data and responders_data ###
len(features_data)
len(responders_data)

In [ ]:
features_data.head(10)
responders_data.head(10)

In [ ]:
## The Problem solving for the dataset starts from here starting with installing the right packages ###
######################################################################################################

In [ ]:
!pip install polars

In [ ]:
""""
import pandas as pd
import polars as pl
train = \
pl.scan_parquet(
    f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet"
).\
select(
    pl.int_range(pl.len(), dtype=pl.UInt64).alias("id"),
    pl.all(),
).collect()

# Read the parquet file and convert to pandas dataframe
# Slice to only get the first 40,000 rows
train_df = train.to_pandas().iloc[:950000]
#In a similar manner i can read the test files
test = \
pl.scan_parquet(
    f"/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet"
).\
select(
    pl.int_range(pl.len(), dtype=pl.UInt64).alias("id"),
    pl.all(),
).collect()
test_df = test.to_pandas().iloc[:950000]

""""

In [ ]:
##reading the training daatset by partitions :
import pandas as pd
train_df = pd.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=5/part-0.parquet')
train_df.head(10)

In [ ]:
#lets read the test dataframe which can be used to make predictions and it will help to understand how can i move forward
#In a similar manner i can read the test files
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

# Define a schema that matches your dataset
schema = pa.schema([
    ('date_id', pa.int32()),  # Set date_id explicitly as int32
    # Add other fields here, matching their expected types from the dataset
])

# Read the parquet file using the defined schema
try:
    test_table = pq.read_table('/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet', schema=schema)
    test_df = test_table.to_pandas()  # Convert to pandas DataFrame
except pa.ArrowInvalid as e:
    print(f"Schema mismatch error: {e}")


In [ ]:
# Try using fastparquet engine
#!pip install fastparquet
test_df = pd.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet', engine='fastparquet')

In [ ]:
test_df.columns

In [ ]:
print(len(train_df))


In [ ]:
len(test_df)

In [ ]:
#Lets understand the train_df 
train_df.describe()

In [ ]:
##lets check the null values in the train df
train_df.isnull().sum()

In [ ]:
#lets train a simple model on to understand how its working
print(train_df.columns)

In [ ]:
import matplotlib.pyplot as plt
missing_values = train_df.isnull().sum()

# Filter columns that have missing values
missing_values = missing_values[missing_values > 0]

# Create a bar plot of missing values
plt.figure(figsize=(19, 6))
missing_values.plot(kind='bar', color='skyblue')
plt.title('Missing Values per Column')
plt.ylabel('Number of Missing Values')
plt.xlabel('Columns')
plt.xticks(rotation=90)
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
print(train_df.columns)

In [ ]:
#let createa heatmap of correlations to understand the feature collinearity within the dataset :
import seaborn as sns
import matplotlib.pyplot as plt

# Drop all responder columns except responder_6
data = train_df.drop(columns=['responder_0', 'responder_1', 'responder_2', 'responder_3', 
                        'responder_4', 'responder_5', 'responder_7', 'responder_8'])

# Set up the figure size for a large heatmap
plt.figure(figsize=(20, 16))

# Generate the heatmap for the dataset correlations
sns.heatmap(data.corr(), annot=False, cmap='coolwarm', linewidths=0.5)

# Add title
plt.title('Heatmap of Correlations for All Columns (Including responder_6)')
plt.show()

In [ ]:
## Training a LGM Model for the training dataset to find out the feature importance by using the mode.This is an initial step in
##learning to get the best fit model and make transition to deep learning model using tensorflow.  #########
!pip install lightgbm

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Step 1: Drop columns with all NaN values
train_df = train_df.dropna(axis=1, how='all')

# Step 2: Split the data into features and target
X = train_df.drop(columns=[ 'date_id', 'time_id',\
                           'feature_39','feature_40', 'feature_43',\
                           'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49',\
                           'feature_62', 'feature_63', 'feature_64','feature_65', 'feature_66',\
                           'responder_3','responder_8','responder_6','responder_7','responder_4','responder_5','responder_2',\
                           'feature_09','feature_06','feature_60','feature_07','feature_15', \
                           'feature_33','feature_34','feature_35','feature_36','feature_37','feature_38','feature_51',\
                           'feature_54','feature_56','feature_57','feature_59',\
                           'feature_61','feature_67','feature_68','feature_69','feature_70','feature_71','feature_72',\
                           'feature_75','feature_76','feature_77','feature_78','feature_05',\
                            'responder_0','responder_1', 'feature_00','feature_01','feature_02','feature_03','feature_04',\
                          'feature_21','feature_26','feature_27',\
                           'feature_31','feature_58','feature_50','feature_17','feature_16','feature_24','feature_12','feature_14','feature_41',\
                          'feature_32','feature_73'])  
# Features
y = train_df['responder_6']  # Target variable
print(X.columns)
# Step 3: Handle missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # Replace NaN values with the mean
X = imputer.fit_transform(X)  # Apply imputer to the features

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 5: Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 6: Initialize and train the SGDRegressor model
model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)
#model = LGBMRegressor(n_estimators=100, learning_rate=0.1, num_leaves=31)
model.fit(X_train_scaled, y_train)

# Step 7: Make predictions
y_pred = model.predict(X_test_scaled)

# Step 8: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")



In [ ]:
#lets see the columsn present within the test_df 
print(test_df.columns)


In [ ]:
#lets prediction the submision.csv file using the above lgbm model and then we can improve further ###
import pandas as pd

import pandas as pd

# Step 1: Drop unnecessary columns in the test dataset
# Columns to drop in test_df based on the columns that were dropped from train_df
columns_to_drop_in_test = ['is_scored','feature_34','feature_05','feature_35', 'feature_36', 'feature_37', 'feature_38',
                           'feature_51', 'feature_54', 'feature_56', 'feature_57', 'feature_59',
                           'feature_61', 'feature_67', 'feature_68', 'feature_69', 'feature_70', 
                           'feature_71', 'feature_72', 'feature_75', 'feature_76', 'feature_77',
                           'feature_78','feature_00', 'feature_01', 'feature_02', 'feature_03','feature_04',\
                          'feature_06', 'feature_07', 'feature_09', 'feature_21','feature_21','feature_26','feature_27',\
                          'feature_31','feature_40','feature_43','feature_45','feature_46','feature_47',\
                          'feature_48','feature_49','feature_60','feature_62','feature_63','feature_64','feature_65',\
                          'feature_66', 'row_id', 'date_id', 'time_id','feature_15','feature_33','feature_39',\
                          'feature_00','feature_01','feature_02','feature_03','feature_04','feature_21','feature_26','feature_27',\
                           'feature_31','feature_58','feature_50','feature_17','feature_16','feature_24','feature_12','feature_14',\
                          'feature_41',\
                          'feature_32','feature_73']

# Drop the columns from test_df that were not used in training
X_test_df = test_df.drop(columns=columns_to_drop_in_test)

# Step 2: Handle missing values in test_df using the same SimpleImputer from train_df
X_test_df = imputer.transform(X_test_df)  # Use the same imputer fitted on the training data

# Step 3: Standardize test_df using the same StandardScaler from train_df
X_test_df_scaled = scaler.transform(X_test_df)  # Use the same scaler fitted on the training data

# Step 4: Make predictions on the test_df using the trained model
y_test_pred = model.predict(X_test_df_scaled)

# Step 5: Create a DataFrame with 'id' column and the predictions
submission_df = pd.DataFrame({
    'row_id': test_df['row_id'],  # Use 'id' column from test_df as identifier
    'prediction': y_test_pred
})

# Step 6: Save the DataFrame as a CSV file for submission to Kaggle
#submission_df.to_csv('submission.csv', index=False)
#print(submission_df)
#print("Submission file 'submission.csv' created successfully!")



In [ ]:
# Step 9: Get feature importance (coefficients)
import matplotlib.pyplot as plt
coefficients = model.coef_

# Step 10: Create a DataFrame to display feature importance
features_df = pd.DataFrame({
    'Feature': train_df.drop(columns=[ 'date_id', 'time_id',\
                           'feature_39','feature_40', 'feature_43',\
                           'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49',\
                           'feature_62', 'feature_63', 'feature_64','feature_65', 'feature_66',\
                           'responder_3','responder_8','responder_6','responder_7','responder_4','responder_5','responder_2',\
                           'feature_09','feature_06','feature_60','feature_07','feature_15', \
                           'feature_33','feature_34','feature_35','feature_36','feature_37','feature_38','feature_51',\
                           'feature_54','feature_56','feature_57','feature_59',\
                           'feature_61','feature_67','feature_68','feature_69','feature_70','feature_71','feature_72',\
                           'feature_75','feature_76','feature_77','feature_78','feature_05',\
                            'responder_0','responder_1', 'feature_00','feature_01','feature_02','feature_03','feature_04',\
                          'feature_21','feature_26','feature_27',\
                           'feature_31','feature_58','feature_50','feature_17','feature_16','feature_24','feature_12','feature_14',\
                                     'feature_41',\
                          'feature_32','feature_73']).columns,
    'Coefficient': coefficients
})

# Step 11: Sort by absolute coefficient value to get the most important features
features_df['Abs_Coefficient'] = features_df['Coefficient'].abs()
top_features = features_df.sort_values(by='Abs_Coefficient', ascending=False)

# Display the sorted feature importance
print(top_features)

# Step 12: Optionally, visualize the top 10 features
top_10_features = top_features.head(30)

plt.figure(figsize=(10, 6))
plt.barh(top_10_features['Feature'], top_10_features['Abs_Coefficient'], color='skyblue')
plt.xlabel('Absolute Coefficient Value (Importance)')
plt.ylabel('Feature')
plt.title('Top 10 Most Important Features in SGDRegressor')
plt.gca().invert_yaxis()  # Invert y-axis to display the most important feature at the top
plt.show()

In [ ]:
# Step 6: Get and print the coefficients (these represent the model weights)
coefficients = model.coef_

# Display the coefficients
print("Model Coefficients (Weights):")
for feature, coef in zip(train_df.drop(columns=['date_id', 'time_id']).columns, coefficients):
    print(f"{feature}: {coef}")

In [ ]:
##Training a tensorflow model to run it on GPU as well instead of just running on TPU as the above model is memory intensive 
##we need to more efficient model and this will definitely work .Tensorflow is generatiev AI framework. ######

In [ ]:
##Finding the relationship of variables selected as features with the target variables ##
